*Glory to Jesus Christ!*
# Financial Calculus: An introduction to derivative pricing.
## Chapter 1: Introduction

### The Parable of the Bookmaker

_Dollar signs omitted where remembered, because they render differently between JupyterLab and GitLab._ 
A bookmaker takes bets on a two-horse race.
A very sharp bookmaker, he correctly calculates that Horse 1's chance of winning is 25%, and Horse 2's chance of winning is 75%.
Hence, the odds are set at 3-1 against Horse 1, and 3-1 on Horse 2.
But, popular sentiment does not reflect the bookmaker's calculations: 5000 are bet on Horse 1, and 10000 on Horse 2.

If Horse 2 were to win, the bookmaker would make a net profit of $5000 - (1/3)\cdot 10000 = 1667 $.
But, if Horse 1 were to win, the bookmaker would make a net profit of $10000 - (3/1) \cdot 5000 = -5000 $. The expected value of his profit is $25\% \cdot (-5000) + 75\% \cdot (1667) \approx 0 $, or exactly even. In the long term, over a number of similar but independent races, the bookmaker would break even. Until then, there is a chance of making a large loss.

Instead, suppose the bookmaker had set odds *according to the wagered money*: that is, 2-1 against and 2-1 on respectively. Our new outcomes are:
$5000 - (1/2) \cdot 10000 = 0$ should Horse 2 win, and $10000 - (2/1) \cdot 5000 = 0 $ should Horse 1 win.

The bookmaker breaks even every time. The outcome is irrelevant. In practice, the bookmaker sells more than 100% of the race, and shortens the odds for profit. However, the same pattern emerges: for the bookmaker to extract a steady riskless income, he assumes the horse's chances from the amounts of money wagered, and not from what they are in reality.

This puts the bookmaker in the surprising position of being uninterested in the outcome of the race. His profit is assured.

In [41]:
using Printf
# Assumes a two-horse race. Use positive numbers, 0 <= j, and 0 < odds1 < 1.

function adjusted_odds(odds1, wager1, wager2, juice) # Can't simplify odds as adding to zero because juice complicates. Add a visual widget later.
    odds2 = 1.0-odds1
    juiced_odds2 = odds2 + juice/2
    juiced_odds1 = odds1 + juice/2
    corrected_odds1 = wager1/(wager1+wager2) + juice/2
    corrected_odds2 = wager2/(wager1+wager2) + juice/2
    profit1 = wager1 + wager2 - wager1/juiced_odds1
    profit2 = wager1 + wager2 - wager2/juiced_odds2
    expected_profit = profit1 * odds1 + profit2 * odds2
    corrected_profit1 = wager1 + wager2 - wager1/corrected_odds1
    corrected_profit2 = wager1 + wager2 - wager2/corrected_odds2
    corrected_expected_profit = corrected_profit1 * odds1 + corrected_profit2 * odds2
    @printf("Quoted probability (horse 1, horse 2): %.2f , %.2f \n", juiced_odds1, juiced_odds2)
    @printf("Profit by horse win (horse 1, horse 2): %.2f , %.2f \n", profit1, profit2)
    @printf("EXPECTED PROFIT following \"true\" quoted probability: %.2f \n \n", expected_profit)
    @printf("Implied probability (horse 1, horse 2): %.2f , %.2f \n", corrected_odds1, corrected_odds2)
    @printf("Profit by horse win (horse 1, horse 2): %.2f , %.2f \n", corrected_profit1, corrected_profit2)
    @printf("EXPECTED PROFIT following wager-implied probabilities: %.2f \n \n", corrected_expected_profit)
end

adjusted_odds(.45, 5000, 10000, .07)

Quoted probability (horse 1, horse 2): 0.48 , 0.59 
Profit by horse win (horse 1, horse 2): 4690.72 , -2094.02 
EXPECTED PROFIT following "true" quoted probability: 959.12 
 
Implied probability (horse 1, horse 2): 0.37 , 0.70 
Profit by horse win (horse 1, horse 2): 1425.34 , 748.22 
EXPECTED PROFIT following wager-implied probabilities: 1052.92 
 


## 1.1: Expectation Pricing

Take for example a coin-flip game. A fair coin is flipped, paying you one dollar for heads and nothing for tails.

Profit follows expectation, which follows probability. In discrete cases, the expectation is the total of each outcome's value weighted by its attached probability. The probability of the coin landing heads or tails is $\frac{1}{2}$. Hence, the expected payoff in the game is:
$$ \frac{1}{2} \cdot 1 + \frac{1}{2} \cdot 0 = 0.50  $$
We link this formal expectation to a 'price' for the game via something like the **Kolmogorov strong law of large numbers**.

>#### Kolmogorov's Strong Law of Large Numbers:
>Suppose there exists a sequence of independent random numbers $X_1, \ldots, X_n$ all sampled from the same distribution with mean $\mu$. We let $S_n$ be the arithmetical average of the sequence up to the $n$th term, i.e.
$$S_n = (X_1 + \ldots + X_n)/n$$
>With probability 1, as $n$ gets larger the value of $S_n$ approaches the mean $\mu$ of the distribution.

The average profit (or loss) per game tends towards the mathematical expectation, less the price paid to play the game. If this difference is positive, then the game is profitable over the long-run (with certainty). If the difference is negative, the player will approach a guaranteed loss with certainty. In this sense, fifty cents is a fair price for the coin-game.

But, in the *short-term*, nothing is guaranteed. The forty-cent price is fair, but not enforceable. Suppose the game was offered for 40-cents on the dollar, instead of fifty cents for a single play. Kolmogorov (*the strong law*) would let the player take advantage of the bookmaker over repeated plays. But if only one play is allowed, then liquidating all of one's belongings to play the game would be irrational, despite it essentially being a source of free money.

So, the 'market' for this game could "trade away" from an expectation-justified price. Any price may be charged for the game in the short-term, and 'buyers' or 'sellers' happy with that price might pay little attention to mathematical expectations of the game's outcome. That said, the strong law works well as a rough starting-point.



### Time Value of Money

Suppose the coin-game paid off not immediately, but one year after the coinflip (and one year after payment was made to play).

If we are in January, one dollar in December is not worth one dollar now; it's worth something less. The formal acknowledgement of this is manifest through *interest rates*, and a bonds market derived from them.

>#### Time Value of Money
>We assume that for any time $T$ less than some time horizon $\tau$, the value now of a dollar promised at time $T$ is given by $e^{-rT}$ for some constant $r>0$. The rate $r$ is then the *continuously compounded* interest rate for this period.

(In real interest-rate markets, $r$ is usually not constant.)

We can derive a strong-law price for the game played at time $T$: paying 50 cents at $T$ is equivalent to paying $50e^{-rT}$ now. Why is this?

By purchasing a half-unit of an appropriate bond at $t=0$ for $50e^{-rT}$ cents, the cost of one coin-flip will be covered at time $T$.

### Stocks

*Most of this chapterette will soon be discredited. Don't worry about it too much.*

Some theorize that stocks are log-normally distributed, i.e.:

>#### Stock Model
>Assume there exists random variable X, which is normally distributed with mean $\mu$ and standard deviation $\sigma$, s.t. the change in the logarithm of the stock price over some period $T$ is given by $X$. In essence:
$$log(S_T) = log(S_0) + X$$ or $$S_T = S_0\cdot e^{X}$$

Next, define a derivative contract known as a **forward**: The seller agrees to give the buyer a unit of stock at some agreed point in the future, in exchange for a sum paid now.

What amount should be written into the contract now, to pay for the stock one year into the future? The answer relies on the time-value of money.

Let $S_T$ represent stock price at time $T$, and let $K$ represent the price written into the contract. At expiry, the naive assumption is that the contract's value is $S_T-K$ (the value of the stock, less the contract's price). Considering the TVM, the value of the claim *now* is $e^{-rT}(S_T-K)$, and the strong law suggests that the expected value should be zero; otherwise, the game would be profitable for only one side (at the expense of the other). If $\mathbb{E}(e^{-rT}(S_T-K)) = 0$, then $K = \mathbb{E}(S_T)$.

To find that, we apply the log-normal stock model above, in combination with **the law of the unconscious statistician**:

>### The Law of the Unconscious Statistician
> For a real-valued random variable X with probability density function $f(x)$, then for any integrable real function $h$, the expectation of $h(X)$ is
$$ \mathbb{E}(h(X)) = \int_{-\infty}^{\infty}{h(x)f(x)dx} $$

and since $X$ is normally distributed, the density function for $X$ is
$$f(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \text{exp}(\frac{-(x-\mu)^2}{2\sigma^2})$$

Integrate to receive $\mathbb{E}(S_T) = S_0\text{exp}(\mu+\frac{1}{2}\sigma^2)$

Time-discounting this expectation gives a theoretical value which looks appealing at first glance.

## 1.2 Arbitrage Pricing

Ultimately, the strong law fails as a form of forward pricing. The contract price can instead be *construced* to yield $S_0e^{rT}$, something that doesn't depend on stock price distribution at all. In short, this is because either counterparty would be capable of buying or selling the contract opportunistically, while guaranteeing a profit.

### Construction Strategy

Sell a unit of the contract, with delivery obliged at $T$. Borrow $S_0$ dollars, and buy with them a stock. At expiry time, this costs the seller $S_0e^{rT}$, the cost of the loan with interest. As they own the stock already, they're not liable for anything else.

As a result, the seller has financial incentive to only sell the contract at a price $\geq S_0e^{rT}$. Else, they would be guaranteed to lose money.

Now, consider the counterparty (a trader buying a unit of the contract): Buy a unit of the contract, with delivery obliged at $T$. Sell a stock now for $S_0$ dollars, and lend it. By expiry time, the loan with interest will be worth $S_0e^{rT}$, and the buyer will once again receive a unit of stock from the contract. If the contract was bought for over $S_0e^{rT}$, then the buyer lost money.

As a result, the buyer has financial incentive to only buy the contract at a price $\leq S_0e^{rT}$. Hence, the ability to trade primitives to compose the contract force the contract price into a narrow band of predictable behavior.

## 1.3 Expectation vs Arbitrage

Until 1973, most people would have used strong-law-based pricing models for all sorts of derivatives, including options. Since 1973 however, the infamous Black-Scholes paper demonstrated how ineffective the strong-law is as a means for option pricing. The strong-law will not reappear again, though *expectation* will be used as a tool for risk-free *construction*.

**All derivatves can be constructed from the underlying**; arbirtrage lurks everywhere.

*Glory forever!*